In [ ]:
import os

current_path = os.getcwd()

project_root = os.path.abspath(os.path.join(current_path, '..', '..'))

In [ ]:
import numpy as np

lbs5 = np.load(project_root+'/output/signal-planting-N/lbs5.npy')
lbs10 = np.load(project_root+'/output/signal-planting-N/lbs10.npy')
lbs20 = np.load(project_root+'/output/signal-planting-N/lbs20.npy')

n_values5 = np.load(project_root+'/output/signal-planting-N/n_values5.npy')
n_values10 = np.load(project_root+'/output/signal-planting-N/n_values10.npy')
n_values20 = np.load(project_root+'/output/signal-planting-N/n_values20.npy')

# take only some specific values for better visualization (adjust if needed)
lbs5 = np.concatenate((lbs5[:-4:4], lbs5[-5::]))
lbs10 = np.concatenate((lbs10[:-4:4], lbs10[-5::]))
lbs20 = np.concatenate((lbs20[:-4:4], lbs20[-5::]))
n_values5 = np.concatenate((n_values5[:-4:4], n_values5[-5::]))
n_values10 = np.concatenate((n_values10[:-4:4], n_values10[-5::]))
n_values20 = np.concatenate((n_values20[:-4:4], n_values20[-5::]))

lbs = np.array([lbs5,lbs10,lbs20])
n_values = np.array([n_values5,n_values10,n_values20])

In [ ]:
import matplotlib.pyplot as plt
from tueplots import bundles, axes
from scipy.optimize import curve_fit
from tueplots import markers

%matplotlib widget

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update(markers.inverted())
plt.rcParams.update(axes.grid())

colors = ['ff7f0e','1f77b4','2ca02c'] 

scale_param = 20000 # used for fitting sigmoids (adjust if needed)
Ns = [500000,1000000,2000000]
labels = ['500,000', '1,000,000', '2,000,000']

def sigmoid(x,a,b,c,d): 
    return d+c/(1.0+np.exp(-a*(x-b)))

# bounds for sigmoid fitting (adjust if needed)
amin = [-10,-1,-4]
amax = [10,6,1]
bmin = [-10,-1,-4]
bmax = [10,6,1]
cmin = [-1,-5,-1]
cmax = [30,3,1]
dmin = [-1,-1,-1]
dmax = [30,3,1]

fig, ax = plt.subplots(figsize=(4, 2))

# use different z-orders for more clarity in plots
# (visual effect to have sigmoid 2,000,000 on top of others at the beginning,
# and then sigmoid 500,000 on top at the end)
zo2 = [3,3,2]
zo3 = [4,3,2]

for i in range(3): # 500,000 -> 1,000,000 -> 2,000,000

    stop = 4 # don't take last values, focus on values close to the sigmoid threshold (adjust if needed)
    y_values = lbs[i][:-stop:] 
    x_values = n_values[i][:-stop:]/scale_param

    t1, t2 = 5, 7 # thresholds for z-order changes (adjust if needed)

    # plot sigmoid
    bds = ([min(y_values)+amin[i],min(x_values)+bmin[i],cmin[i],dmin[i]],[max(y_values)+amax[i],max(x_values)+bmax[i],cmax[i],dmax[i]])
    popt, pcov = curve_fit(sigmoid, x_values, y_values, method='dogbox', bounds=bds)
    x_fit = np.linspace(min(x_values), max(x_values))
    y_fit = np.minimum(sigmoid(x_fit, *popt),1) # clip <= 1
    # artificially define a point (0,-0.015) for visual effect (sigmoid plot begin before 0)
    x_fit = np.concatenate((np.array([0]),x_fit))
    y_fit = np.concatenate((np.array([-0.015]),y_fit))
    # z-order 1
    ax.plot((x_fit/Ns[i])[:t1+10], y_fit[:t1+10], label=labels[i],linewidth=3.5,zorder=1,color='#'+colors[i])
    # z-order 2
    ax.plot((x_fit/Ns[i])[t1+9:t2+15], y_fit[t1+9:t2+15],linewidth=3.5,zorder=zo2[i],color='#'+colors[i])
    # z-order 3
    ax.plot((x_fit/Ns[i])[t2+14:], y_fit[t2+14:],linewidth=3.5,zorder=zo3[i],color='#'+colors[i])

    # scatter points
    # z-order 1
    ax.scatter((x_values/Ns[i])[:t1],y_values[:t1],marker='o',edgecolor='#'+colors[i],color='white',linewidth=1,zorder=2+1,s=50)
    # z-order 2
    ax.scatter((x_values/Ns[i])[t1:t2],y_values[t1:t2],marker='o',edgecolor='#'+colors[i],color='white',linewidth=1,zorder=zo2[i]+1,s=50)
    # z-order 3
    ax.scatter((x_values/Ns[i])[t2:-1],y_values[t2:-1],marker='o',edgecolor='#'+colors[i],color='white',linewidth=1,zorder=zo3[i]+1,s=50)

# ticks
ax.tick_params(axis='both', labelsize=10)
xticks_major = np.arange(0, 31, 2) / (100 * scale_param)
ax.set_xticks(xticks_major)
xticks_minor = np.arange(1, 30, 2) / (100 * scale_param)
ax.set_xticks(xticks_minor, minor=True)
new_tick_labels = [f'{round(tick * scale_param * 100)}' for tick in xticks_major]
ax.set_xticklabels(new_tick_labels)
ax.tick_params(axis='x', which='major', length=7)
ax.tick_params(axis='x', which='minor', length=4)
ax.tick_params(axis='x', which='both', width=1)
ax.set_xlim(0, 1.5e-5)
ax.set_yticks(np.arange(0, 101, 5)/100, minor=True) 
ax.set_yticks(np.arange(0, 101, 20)/100, minor=False) 
ax.set_yticklabels(np.arange(0, 101, 20)/100, minor=False)
ax.tick_params(axis='y', which='major', length=7)
ax.tick_params(axis='y', which='minor', length=4) 
ax.tick_params(axis='y', which='both', width=1) 
ax.set_ylim(-0.05,1.05)

ax.grid()

fig.supylabel('Success', fontsize=12, x=0.01, y=0.6)
fig.supxlabel(r'Relative collective size $n/N$ (in $\%$)', fontsize=12)
ax.legend(loc='lower right', fontsize=9)

plt.tight_layout()
plt.subplots_adjust(bottom=0.3, wspace=0.3)  # ensure space for labels

# force spines to be drawn with higher z-order so that points are not on top of axes spines
for spine in ax.spines.values():
    spine.set_zorder(10) 

plt.savefig(project_root+"/plots/signal-planting-N.pdf", format="pdf", bbox_inches="tight")

plt.show()